In [1]:
%cd ..

C:\MY FILES\CFE_VIDEO_MEMBERSHIP_WEB_APP


In [2]:
from app import db
from app.users.models import User
from cassandra.cqlengine.management import sync_table
from pydantic import BaseModel, EmailStr, SecretStr, field_validator

In [3]:
db.get_session()
sync_table(User)

In [4]:
user_data = {
    "email": "abc@abc.com",
    "password": "abc123"
}

In [5]:
class UserLoginSchema(BaseModel):
    email: EmailStr
    password: SecretStr

In [6]:
UserLoginSchema(**user_data)

UserLoginSchema(email='abc@abc.com', password=SecretStr('**********'))

In [13]:
class UserSignupSchema(BaseModel):
    email: EmailStr
    password: SecretStr
    password_confirm: SecretStr

    @field_validator("password_confirm")
    def passwords_match(cls, v, values, **kwargs):
        password = values.data["password"]
        password_confirm = v
        if password != password_confirm:
            raise ValueError("Password doesn't match!")
        return v

    @field_validator("email")
    def email_available(cls, v, values, **kwargs):
        q = User.objects().filter(email=v).allow_filtering()
        if q.count() != 0:
            raise ValueError("Email is not available.")
        return v

In [17]:
UserSignupSchema(email="md.sadat.amin2@gmail.com", password="abc123", password_confirm="abc123")

UserSignupSchema(email='md.sadat.amin2@gmail.com', password=SecretStr('**********'), password_confirm=SecretStr('**********'))